In [1]:
from pyspark import SparkContext
sc =SparkContext()

In [86]:
numPartitions = 10

lines = sc.textFile('./adj_noun_pairs.txt', numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()
# pairs.collect()

PythonRDD[152] at RDD at PythonRDD.scala:48

In [89]:
def f(iterator):
    for i in iterator:
        if i[1] == "unification": 
            yield i[0]
pairs.mapPartitions(f).first()

u'several'

In [85]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions).cache()
def gf(index,s):
    flag = 0
    for p in s:
        if p[0] < p[1]:
            flag = -1
            break
        elif p[0] == p[1]:
            continue
        else:
            flag = 1
            break
    yield (index,flag)
r = rdd.mapPartitionsWithIndex(gf).collect()
r_dict =dict(r)
flag = '=='
for i in range(len(r)):
    if r_dict[i] < 0:
        flag = '<'
        break
    if r_dict[i] > 0:
        flag =  '>'
        break
print(flag)

<


### Prefix Sums

In [44]:
x = [1, 4, 3, 5, 6, 7, 0, 1]

rdd = sc.parallelize(x, 4).cache()

def f(iterator):
    s = 0
    for i in iterator:
        s += i
    yield s

sums = rdd.mapPartitions(f).collect()  # 每个partition 执行

print sums

for i in range(1, len(sums)):
    sums[i] += sums[i-1]

def g(index, iterator):
    global sums
    if index == 0:
        s = 0
    else:
        s = sums[index-1]
    for i in iterator:
        s += i
        yield s

prefix_sums = rdd.mapPartitionsWithIndex(g)  # 传入index
print prefix_sums.collect()

[5, 8, 13, 1]
[1, 5, 8, 13, 19, 26, 26, 27]


### Monotocity checking单调性检验

In [2]:
x = [1, 3, 4, 5, 7, 3, 10, 14, 16, 20, 21, 24, 24, 26, 27, 30]

rdd = sc.parallelize(x, 4).cache()

def f(iterator):
    first = iterator.next()
    last = first
    increasing = True
    for i in iterator:
        if i < last:
            increasing = False
        last = i
    yield increasing, first, last

results = rdd.mapPartitions(f).collect()

print results

increasing = True
if results[0][0] == False: # 第一个处理
    increasing = False
else:
    for i in range(1, len(results)): #从1开始
        if results[i][0] == False or results[i][1] < results[i-1][2]: # 当前的first小于前一个的last，False
            increasing = False
if increasing:
    print "Monotone"
else:
    print "Not monotone"


[(True, 1, 5), (False, 7, 14), (True, 16, 24), (True, 24, 30)]
Not monotone


### Maximum Subarray Problem

In [3]:
# Classical divide and conquer algorithm

import sys

A = [-3, 2, 1, -4, 5, 2, -1, 3, -1]

def MaxSubarray(A, p, r):
    if p == r:
        return A[p]
    q = (p+r)/2
    M1 = MaxSubarray(A, p, q)
    M2 = MaxSubarray(A, q+1, r)
    Lm = -sys.maxint
    Rm = Lm
    V = 0
    for i in range(q, p-1, -1):
        V += A[i]
        if V > Lm:
            Lm = V
    V = 0
    for i in range(q+1, r+1):
        V += A[i]
        if V > Rm:
            Rm = V
    return max(M1, M2, Lm+Rm)

print MaxSubarray(A, 0, len(A)-1)

9


In [5]:
# Linear-time algorithm
# Written in a way so that we can call it for each partition

def linear_time(it):
    Vmax = -sys.maxint
    V = 0
    for Ai in it:
        V += Ai
        if V > Vmax:
            Vmax = V
        if V < 0:
            V = 0
    yield Vmax
    
print linear_time(A).next()

9


In [8]:
# The Spark algorithm:

def compute_sum(it):
    yield sum(it)

def compute_LmRm(index, it):
    Lm = -sys.maxint
    Rm = -sys.maxint
    L = sums[index]
    R = 0
    for Ai in it:
        L -= Ai
        R += Ai
        if L > Lm:
            Lm = L
        if R > Rm:
            Rm = R
    yield (Lm, Rm)

num_partitions = 4
rdd = sc.parallelize(A, num_partitions).cache()
sums = rdd.mapPartitions(compute_sum).collect()
print sums
LmRms = rdd.mapPartitionsWithIndex(compute_LmRm).collect()
print LmRms
best = max(rdd.mapPartitions(linear_time).collect())

for i in range(num_partitions-1):
    for j in range(i+1, num_partitions):
        x = LmRms[i][0] + sum(sums[i+1:j]) + LmRms[j][1]
        if x > best:
            best = x

print best

[-1, -3, 7, 1]
[(2, -1), (0, 1), (2, 7), (2, 2)]
9
